In [ ]:
#r "nuget: MediaToolkit"
#r "nuget: MetadataExtractor"
using MediaToolkit.Model;
using MediaToolkit;
using MetadataExtractor;
using System.IO;
using Newtonsoft.Json;

In [ ]:
struct FileGroups
{
    /// <summary>
    /// 压缩包的ID
    /// </summary>
    public int GroupId { get; init; }
    /// <summary>
    /// 压缩包的总大小(含分卷)
    /// </summary>
    public long Length { get; init; }
    /// <summary>
    /// 压缩包使用的分卷数
    /// </summary>
    public long VolumeCount { get; init; }
    /// <summary>
    /// 压缩包是单独的(未分卷)
    /// </summary>
    public bool IsSingle { get; init; }

    /// <summary>
    /// 文件的元数据
    /// </summary>
    public struct FileMeta
    {
        public string FullName { get; init; }
        public long Length { get; init; }
        public DateTime WriteTime { get; init; }
        public DateTime CreationTime { get; init; }
        public DateTime AccessTime { get; init; }
        /// <summary>
        /// MetadataExtractor 读取的图片元数据
        /// </summary>
        public Dictionary<string, string>? ImageExif { get; init; }
        /// <summary>
        /// FFMPEG 读取的视频元数据
        /// </summary>
        public Metadata? VideoMeta { get; init; }
        public FileMeta(FileInfo fileinfo)
        {
            // 读取基本数据
            FullName = fileinfo.FullName;
            Length = fileinfo.Length;
            WriteTime = fileinfo.LastWriteTime;
            CreationTime = fileinfo.CreationTime;
            AccessTime = fileinfo.LastAccessTime;

            // 读取图片元数据
            ImageExif = GetImageExif(fileinfo);
            // 读取视频元数据
            VideoMeta = GetVideoMeta(fileinfo);
        }
        /// <summary>
        /// 读取图片元数据
        /// </summary>
        /// <param name="fileinfo"></param>
        /// <returns></returns>
        private static Dictionary<string, string>? GetImageExif(FileInfo fileinfo)
        {
            var exifData = new Dictionary<string, string>();

            try
            {
                var directories = ImageMetadataReader.ReadMetadata(fileinfo.FullName);

                foreach (var directory in directories)
                {
                    foreach (var tag in directory.Tags)
                    {
                        if (tag.Description == null) continue;
                        exifData[tag.Name] = tag.Description;
                    }
                }
            }
            catch { return null; }

            return exifData;
        }
        /// <summary>
        /// 读取视频元数据
        /// </summary>
        /// <param name="fileinfo"></param>
        /// <returns></returns>
        private static Metadata? GetVideoMeta(FileInfo fileinfo)
        {
            try
            {
                var mediaFile = new MediaFile { Filename = fileinfo.FullName };

                using (var engine = new Engine())
                {
                    engine.GetMetadata(mediaFile);
                }
                return mediaFile.Metadata;
            }catch { return null; }
        }
    }
}

In [ ]:
var filemeta = new FileGroups.FileMeta(new FileInfo(@"I:\Camera\_MG_4239.CR3"));

In [ ]:
filemeta

In [ ]:
var str = JsonConvert.SerializeObject(filemeta, Formatting.Indented);
str